# Zoom grader for Biosem

Unlike Piazza-grader, doesn't work with Piazza, but looks at google forms with questions.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import time

In [ ]:
folder_name = '../../data/attendance/'
target_meeting = 'Biosem' # Options: {'Biosem', 'Comput', 'Neuro'}
chat_mask = 'Biosem'

# Part 1. Zoom logs

In [ ]:
# Read attendance data

df = pd.DataFrame()
file_list = os.listdir(folder_name)
print("Total files: ",len(file_list))

processed_set = set()
processed_dict = {}
for fname in file_list:
    if fname[-3:]!='csv':
        continue # Ignore everything that is not a csv
    fullname = folder_name + fname
    header = pd.read_csv(fullname, header=0, nrows=1) # Attempt to read the header
    if 'Topic' not in header.columns: # Not a zoom log, skip this one
        continue
    name = header['Topic'][0]    
    datestring = header['Start Time'][0][:10]
    if name+datestring in processed_set:
        print(f"Duplicated meeting: {name+datestring} ({fname}) already saved as {processed_dict[name+datestring]}")
        continue  # This meeting
    processed_set.add(name+datestring)
    processed_dict[name+datestring] = fname
    
    data = pd.read_csv(fullname, header=2)
    data['Date'] = datestring
    data['Meeting'] = name
    data['User Email'] = data['User Email'].fillna('none') # NaNs are ignored by aggregation below
    
    # Join Time	Leave Time	Duration (Minutes)
    if 'Join Time' in data.columns:  # Wrong output format - it is sometimes unavoidable
        data['Total Duration (Minutes)'] = data['Duration (Minutes)']
        data = data.drop(columns=['Join Time', 'Leave Time', 'Duration (Minutes)'])
    
    df = df.append(data, ignore_index=True)

print('Meeting files:', len(processed_set))

In [ ]:
# Rename fields, standardize meetings that had synonyms

df= df.rename({'User Email': 'email', 
               'Total Duration (Minutes)': 'minutes', 
               'Name (Original Name)': 'name',
               'Meeting':'meeting',
               'Date':'date'}, axis=1)
meeting_dict = {'Biosem Zoom': 'Biosem', 
                'Biosem_spring_2021': 'Biosem',
                'Neuroscience': 'Neuro',
                'Computational Neuro': 'Comput'}
df['meeting'] = df['meeting'].replace(meeting_dict)
df.name = df.name.str.title()  # Capitalize names (for consistency)

In [ ]:
# Inexplicably, some total participation counts are stored as strings, not numbers. Correct that.

df.loc[df.minutes=='Yes'] = 0 # No idea what it means, but Zoom outputed it!
df.minutes = df.minutes.astype(int)
df = df[df.minutes>0]  # Remove weird empty entries

# Full attendance and late classes:
df['check'] = 0.3*(df.minutes>30) + (1-0.3)*(df.minutes>40)

In [ ]:
# What meetings are even there?
set(df.meeting)

In [ ]:
# Participation length histogram
if False:
    plt.figure(figsize=(10,3))
    plt.subplot(121)
    plt.hist(df.loc[df.meeting=='Biosem'].minutes.values, bins=50);
    plt.title('Biosem');
    plt.subplot(122)
    plt.hist(df.loc[df.meeting=='Neuro'].minutes.values, bins=50);
    plt.title('Neuro');

In [ ]:
# Only analyze one type of meetings starting from here
df = df[df.meeting==target_meeting]

In [ ]:
# Read the actual list of students. Note that the file name needs to match the meeting name.

people = pd.read_csv(folder_name + target_meeting + '.csv', header=0, sep=',')
people.columns = [' '.join(s.split()) for s in people.columns] # Remove repeating spaces, just in case
# print(people.columns)
people['name'] = people['FIRST NAME'] + ' ' + people['LAST NAME']
people = (people
          .drop_duplicates() # A left-over from the "All meetings at once" pipeline, but let's keep for a while
          .reset_index()
          .rename({'ID#':'id', 'Email': 'email'}, axis=1)
          .loc[:,['id', 'email', 'name']]
         )
print(people.columns)
# people.head()

In [ ]:
# Troubleshooting: A full list of meetings with their attendance.
# Check if the plot makes sense.

dfm = df.groupby(['date','meeting','name']).agg({'minutes': sum})
dfm.minutes = 1*(dfm.minutes>10)
dfm = dfm.groupby(['date', 'meeting']).agg({'minutes': sum}).reset_index()
dfm = dfm.rename({'minutes':'people'}, axis=1)
dfm.date = pd.to_datetime(dfm.date)
print("Meetings so far:", dfm.shape[0])

plt.figure(figsize=(9,2))
plt.plot(dfm.date.dt.dayofyear, dfm.people, '.');
plt.xlabel('Meeting');
plt.ylabel('People');

# print(dfm)

In [ ]:
# Check if there are any entries without email at all
df[[len(a)<10 for a in df.email]]

In [ ]:
# Read and process known alts

df_alts = pd.read_csv(folder_name + 'alts.csv', header=0, sep=',')

# First find official emails where available, and drop some users manually (e.g. the instructor)
df_fixed = (df
            .merge(df_alts, on='email', how='left', suffixes=['', '_r'])
            .drop(columns=['name_r'])
            .query("real_email != 'none'")
           )

# Substitute 'wrong' emails with 'official' emails where available
ind = (df_fixed.real_email.notna())
df_fixed.loc[ind, 'email'] = df_fixed.loc[ind, 'real_email']
df_fixed = df_fixed.drop(columns='real_email')

In [ ]:
# Find people with emails that aren't on the official list
# Output them, to manually create a list of alt-emails (that will be )

df_lost = (df_fixed           
           .groupby(['name','email'])
           .agg({'minutes': sum})
           .reset_index()
           .merge(people, on='email', how='left', suffixes=['', '_r']) # Ignore official name (_r)
           .drop(columns='name_r')           
           .query('id.isna()', engine='python') # Only keep unrecognized users
          )
print('Unrecognized emails (uncomment to see the list):', len(df_lost.id))

# Output nice human-readable form:
df_lost

In [ ]:
# Real name searher:
token='jerrdd'
df_fixed.query('name.notna()', engine='python').query('name.str.lower().str.contains(@token)', engine='python')

In [ ]:
# df_fixed.fillna(' ').query("email.str.contains('6406')", engine='python')

In [ ]:
# Lookup correct names, and drop all users who are not on bip
df_fixed = (df_fixed
            .merge(people, on='email', how='left', suffixes=['_l', '']) # This time keep official names only
            .drop(columns=['name_l', 'id', 'Guest'])
           )

In [ ]:
# Build a summary

dfs = (df_fixed
       .groupby(['name','email'])
       .agg({'check': sum})
       .reset_index()
       .sort_values(by='check')
      )

#dfs

# Part 2: Google responses

In [ ]:
full_folder_name = folder_name + 'biosem_questions/'

df_responses = pd.DataFrame()
file_list = os.listdir(full_folder_name)
print("Total files: ",len(file_list))

for fname in file_list:    
    fullname = full_folder_name + fname    
    date = ' '.join(fullname.split(' ')[2:4])[:-1] + ' 2021, 12:10 pm'
    # print(date)
    
    df_temp = pd.read_excel(fullname).iloc[:,:-1]
    df_temp.columns = ['timestamp', 'email']
    df_temp = (df_temp
               .groupby('email')
               .agg({'timestamp':'min'})
               .reset_index()
              )
    df_temp['date'] = date
    df_responses = df_responses.append(df_temp)
    
#print('Total lines:', df_responses.shape[0])
df_responses.date = pd.to_datetime(df_responses.date)
df_responses['good'] = df_responses.timestamp < df_responses.date
df_responses['late'] = (df_responses.timestamp - df_responses.date)
df_responses.email = df_responses.email.str.lower()
df_responses.late = np.maximum(0, df_responses.late.dt.days*24*60 + np.round(df_responses.late.dt.seconds/60)).astype(int)

df_responses = (df_responses
                .reset_index()
                .drop(columns='index')
               )
#df_responses.head(5)

ind = np.argwhere(df_responses.email.str[-4:].str.match('bard').values)[0]
df_responses.loc[ind, 'email'] = df_responses.email[ind].str.replace('bard', 'bard.edu')
df_responses.email = df_responses.email.str.replace('gmail.com', 'bard.edu')

In [ ]:
# Unrecognized emails
(df_responses
 .groupby('email')
 .agg({'good':'sum'})
 .reset_index()
 .merge(dfs, on='email', how='left')
 .query("check.isna()", engine='python')
)

In [ ]:
# Questions that were submitted late
(df_responses
 .query('not good') 
 .merge(dfs[['email','name']], on='email')
)

# Final summary grading

In [ ]:
# Full output

df_full = (df_responses
           .groupby('email')
           .agg({'good':'sum'})
           .merge(dfs, on='email') 
           .assign(passed = (df_full.check>=13-1-3) & (df_full.good >= 13-2-3))           
          )
# No info about RNA seminar, so -1
# First and last seminar there were no questions, so -2
# 3 seminars can be missed, so -3 in both cases

# Show the full giant thing:
with pd.option_context('display.max_rows', 14000, 'display.width', 1000):
    # print(df_full)
    pass

df_full.sort_values('passed')

In [ ]:
# Students who didn't submit enough questions
df_full.query('good<13-2-3')

# Analyze one student

In [ ]:
# Check individual records of needed
token = 'Cree'

# Full attendance:
all_meetings = df.groupby('date').agg({'name':'count'}).reset_index()
partial = df.loc[df.name.str.contains(token)]
out = all_meetings[['date']].merge(partial, on='date', how='left')[['date','name','email','minutes','check']]
out.loc[out.check.isna(),'check'] = 0
print(sum(out.check))

out

In [ ]:
# All homeworks:
df_responses[df_responses.email.str.contains('3431')].sort_values('date')

# Mid-term letters

In [ ]:
def response(df_full, df_piazza, i):
    # Write a full response for one student
    
    sec = df_full.iloc[i]
    name = sec.name_piazza
    
    flag_submitted_something = False
    if name in df_piazza.name.unique():
        flag_submitted_something = True
        missing_work = (df_piazza
                        .groupby(['work', 'name'])
                        .agg({'message':'count'})
                        .reset_index()
                        .pivot_table(index='work', columns='name', values='message')
                        .reset_index()
                        [['work', name]]
                        .set_axis(['work', 'name'], axis=1, inplace=False)
                        .query('name.isna()', engine='python')
                        .work
                        .sort_values()
                        .tolist()
                     )
    
    def remove_double_spaces(s):
        s = s.replace('\n', ' ')
        return ' '.join([c for c in s.split(' ') if c])
    
    print(f"{sec.name_zoom} {sec.email}:")
    print(f"Dear {sec.name_piazza.split()[0]},")
    print(f"based on my records as of {max(df.date)}")
    print(f"you have attended {np.round(sec.check).astype(int)} classes ", end='')
    print(f"(out of {classes_so_far} that we had so far), ")
    print(f"and submitted {sec.total} homeworks (out of {works_so_far} that we had)")   
    if sec.good>0:
        print(f"{sec.good} of these homeworks were really good!! (and others were also fine :)")
    if sec.total < works_so_far and flag_submitted_something:
        print(f"\n(Works that are missing: {'; '.join(missing_work)})\n")
    print(f"You were active in {sec.n_chats} class sessions", end='')
    if sec.percentile_chats<50:
        print(f", which places you in the top {sec.percentile_chats}%", end='')
    else:
        print(f", which places you in the bottom {100-sec.percentile_chats}%", end='')
    print(" in terms of class participation.")
    
    print(f"Taking all of it into account, your current automatically projected grade is ", end='')
    if sec.grade=='B': print('between B and B+')
    elif sec.grade=='B+': print('between B+ and A-')        
    else: print(sec.grade)
    
    if sec.grade in {'F', 'C'}:
        message = """If you want to improve, you can submit missing homeworks. 
              For missed lab work, just do the lab, and submit your work, as described in the assignment. 
              For missed questions, at this point the assignment is a bit different.
              You will need to answer the following question: 
              'What is the most interesting thing that you learned that week, and what makes it interesting for you?'. 
              I would expect about a paragraph of text, with explicit references to what you learned from 
              home assignments, and from our class discussions. Additional sources are also welcome!"""
        print('\n'+remove_double_spaces(message))
    print("\n(If anything about this summary does not ring right to you, please email me, " \
          "and I'll double-check my records!)")
        
response(df_full, df_piazza, 12)

In [ ]:
i = np.where(df_full.name_piazza.str.match('Ste'))[0][0]
print(i)
response(df_full, df_piazza, i)

In [ ]:
for i in range(df_full.shape[0]):
    response(df_full, df_piazza, i)
    print("\n\n")

In [ ]:
# Ouput the summary csv
#recognized.to_csv(folder_name + '../attendance_summary.csv', index=False)